# Coursera Data Science IBM Specialization Certificate - Assignment 2

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

Start by creating a new Notebook for this assignment.
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [28]:
#Import required libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests
from collections import namedtuple

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

res = requests.get(url)

if res.status_code == 200:
    html = res.content

soup = BeautifulSoup(html, 'lxml')

In [11]:
table = soup.find('table')

headers = table.find_all('th')
rows = table.find_all('tr')
headers

[<th>Postcode</th>, <th>Borough</th>, <th>Neighbourhood
 </th>]

In [23]:
test = rows[0]
test.find_all("td") == True

False

In [29]:
Neighborhood = namedtuple("Neighborhood", "PostalCode Borough Neighborhood")

In [44]:
def process_row(row):
    items = row.find_all('td')
    out = [item.text.strip() for item in items]
    return out

def process_all_rows(rows):
    processed = []
    for row in rows:
        processed.append(process_row(row))
    return processed

def convert_to_dataset(items):
    output = []
    for item in items:
        if item:
            PostalCode, Borough, Nbh = item
            nb = Neighborhood(PostalCode, Borough, Nbh)
            output.append(nb)
    return output
            
            
    

In [47]:
items = process_all_rows(rows)
dataset = convert_to_dataset(items)
dataset = list(filter(lambda x: x.Borough != 'Not assigned', dataset))

[Neighborhood(PostalCode='M3A', Borough='North York', Neighborhood='Parkwoods'),
 Neighborhood(PostalCode='M4A', Borough='North York', Neighborhood='Victoria Village'),
 Neighborhood(PostalCode='M5A', Borough='Downtown Toronto', Neighborhood='Harbourfront'),
 Neighborhood(PostalCode='M5A', Borough='Downtown Toronto', Neighborhood='Regent Park'),
 Neighborhood(PostalCode='M6A', Borough='North York', Neighborhood='Lawrence Heights'),
 Neighborhood(PostalCode='M6A', Borough='North York', Neighborhood='Lawrence Manor'),
 Neighborhood(PostalCode='M7A', Borough="Queen's Park", Neighborhood='Not assigned'),
 Neighborhood(PostalCode='M9A', Borough='Etobicoke', Neighborhood='Islington Avenue'),
 Neighborhood(PostalCode='M1B', Borough='Scarborough', Neighborhood='Rouge'),
 Neighborhood(PostalCode='M1B', Borough='Scarborough', Neighborhood='Malvern'),
 Neighborhood(PostalCode='M3B', Borough='North York', Neighborhood='Don Mills North'),
 Neighborhood(PostalCode='M4B', Borough='East York', Neighbo

In [92]:
def consolidate_neighborhoods(subset):
    postalcode = subset[0]
    borough = subset[1]["Borough"].unique()[0]
    neighborhoods = ",".join(subset[1]['Neighborhoods'])
    return postalcode, borough, neighborhoods
    

def transform_dataset(dataset):
    pre_postal_codes = []
    pre_boroughs = []
    pre_neighborhoods = []
    
    for item in dataset:
        pre_postal_codes.append(item.PostalCode)
        pre_boroughs.append(item.Borough)
        if item.Neighborhood == 'Not assigned':
            nbh = item.Borough
        else:
            nbh = item.Neighborhood
        pre_neighborhoods.append(nbh)
        
        
    operation = {"PostalCode": pre_postal_codes,  "Borough": pre_boroughs, "Neighborhoods": pre_neighborhoods}
    pre_df = pd.DataFrame(operation).groupby("PostalCode")
    
    postal_codes = []
    boroughs = []
    neighborhoods = []
    
    for sub in pre_df:
        pc, br, nbh = consolidate_neighborhoods(sub)
        postal_codes.append(pc)
        boroughs.append(br)
        neighborhoods.append(nbh)
    
    data = {"PostalCode": postal_codes,  "Borough": boroughs, "Neighborhoods": neighborhoods}
    df = pd.DataFrame(data)
    
    return df

In [93]:
x = transform_dataset(dataset)

x

,PostalCode,Borough,Neighborhoods
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [79]:
x.shape

(211, 3)